## Reference
+ Bank for International Settlements :https://data.bis.org/topics/CBS
+ Economic Networks Theory and Computation: https://github.com/QuantEcon/quantecon-book-networks

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.options.display.max_columns = None
pd.options.display.max_rows = 1200

## Clean data

In [2]:
df_raw = pd.read_csv('WS_CBS_PUB_csv_col.zip', compression='zip', header=0)

In [3]:
df = df_raw.loc[df_raw['CBS bank type'] == 'Domestic banks(4B), excl. domestic positions', :]
df = df.loc[df['CBS reporting basis'] == 'Immediate counterparty basis', :]
df = df.loc[df['Balance sheet position'] == 'Total claims', :]
df = df.loc[df['Collection Indicator'] == 'End of period', :]

df = df.loc[df['Reporting country'] != 'All reporting countries', :]
df = df.loc[df['Counterparty country'] != 'All reporting countries', :]


## clean other countries
# otherlist = ['All countries (total)', 'Unallocated location', 'All countries excluding residents', 'Euro area', 
#              'Netherlands Antilles', 'Residual West Indies UK', 'International organisations', 'Residents/Local',
#              'Residual British Overseas Territories', 'Czechoslovakia', 'German Democratic Republic',
#              'USSR (Soviet Union)', 'U.S. Miscellaneous Pacific Islands'            ]
# df = df.loc[~df['Counterparty country'].isin(otherlist), :]

## countries:
# 'Luxembourg', 'India', 'Norway', 'Singapore' are NaN all the time
countrylist = list(df['Reporting country'].unique())
countrylist = list(set(countrylist) - set(['Hong Kong SAR', 'Chinese Taipei', 'Luxembourg', 'India', 'Norway', 'Singapore'])) 
df = df.loc[df['Counterparty country'].isin(countrylist), :]
df = df.loc[df['Reporting country'].isin(countrylist), :]

In [4]:
quarters0 = ['1983-Q4', '1984-Q2', '1984-Q4', '1985-Q2', '1985-Q4', '1986-Q2', '1986-Q4',
            '1987-Q2', '1987-Q4', '1988-Q2', '1988-Q4', '1989-Q2', '1989-Q4', '1990-Q2',
            '1990-Q4', '1991-Q2', '1991-Q4', '1992-Q2', '1992-Q4', '1993-Q2', '1993-Q4',
            '1994-Q2', '1994-Q4', '1995-Q2', '1995-Q4', '1996-Q2', '1996-Q4', '1997-Q2',
            '1997-Q4', '1998-Q2', '1998-Q4', '1999-Q2', '1999-Q4']
quarters1 = ['2000-Q1', '2000-Q2', '2000-Q3', '2000-Q4', 
            '2001-Q1', '2001-Q2', '2001-Q3', '2001-Q4',
            '2002-Q1', '2002-Q2', '2002-Q3', '2002-Q4',
            '2003-Q1', '2003-Q2', '2003-Q3', '2003-Q4',
            '2004-Q1', '2004-Q2', '2004-Q3', '2004-Q4',
            '2005-Q1', '2005-Q2', '2005-Q3', '2005-Q4',
            '2006-Q1', '2006-Q2', '2006-Q3', '2006-Q4',
            '2007-Q1', '2007-Q2', '2007-Q3', '2007-Q4',
            '2008-Q1', '2008-Q2', '2008-Q3', '2008-Q4',
            '2009-Q1', '2009-Q2', '2009-Q3', '2009-Q4',
            '2010-Q1', '2010-Q2', '2010-Q3', '2010-Q4',
            '2011-Q1', '2011-Q2', '2011-Q3', '2011-Q4',
            '2012-Q1', '2012-Q2', '2012-Q3', '2012-Q4',
            '2013-Q1', '2013-Q2', '2013-Q3', '2013-Q4',
            '2014-Q1', '2014-Q2', '2014-Q3', '2014-Q4',
            '2015-Q1', '2015-Q2', '2015-Q3', '2015-Q4',
            '2016-Q1', '2016-Q2', '2016-Q3', '2016-Q4',
            '2017-Q1', '2017-Q2', '2017-Q3', '2017-Q4',
            '2018-Q1', '2018-Q2', '2018-Q3', '2018-Q4',
            '2019-Q1', '2019-Q2', '2019-Q3', '2019-Q4',
            '2020-Q1', '2020-Q2', '2020-Q3', '2020-Q4',
            '2021-Q1', '2021-Q2', '2021-Q3', '2021-Q4',
            '2022-Q1', '2022-Q2', '2022-Q3', '2022-Q4',
            '2023-Q1', '2023-Q2', '2023-Q3', '2023-Q4']
quarters = quarters0 + quarters1
dict_data = {}
dict_data2 = {}
for qt in quarters:
    df_one = df[['Reporting country', 'Counterparty country', qt]]
    df_one_wide = df_one.pivot(index='Reporting country', columns='Counterparty country', values=qt)
    df_one_wide = df_one_wide.fillna(0)
    dict_data[qt] = df_one_wide
                                      
    df_one = df[['L_REP_CTY', 'L_CP_COUNTRY', qt]]
    df_one_wide = df_one.pivot(index='L_REP_CTY', columns='L_CP_COUNTRY', values=qt)
    df_one_wide = df_one_wide.fillna(0)
    dict_data2[qt] = df_one_wide

In [5]:
crosswalk = df[['Reporting country', 'L_REP_CTY']].drop_duplicates()

## Plot network graphs

In [6]:
from scipy.stats import beta
def to_zero_one_beta(x, 
                qrange=[0.25, 0.75], 
                beta_para=[0.5, 0.5]):
    
    """
    Nonlinearly map vector x to the zero one interval with beta distribution.
    https://en.wikipedia.org/wiki/Beta_distribution
    """
    x = np.array(x)
    x_min, x_max = x.min(), x.max()
    if beta_para != None:
        a, b = beta_para
        return beta.cdf((x - x_min) /(x_max - x_min), a, b)
    else:
        q1, q2 = qrange
        return (x - x_min) * (q2 - q1) /(x_max - x_min) + q1

In [7]:
def plot_graph(A, 
               X,
               ax,
               codes,
               node_color_list=None,
               node_size_multiple=0.0005, 
               edge_size_multiple=14,
               layout_type='circular',
               layout_seed=1234,
               tol=0.03):  # clip entries below tol

    G = nx.DiGraph()
    N = len(A)

    # Add nodes, with weights by sales of the sector
    for i, w in enumerate(X):
        G.add_node(codes[i], weight=w, name=codes[i])

    node_sizes = X * node_size_multiple

    # Position the nodes
    if layout_type == 'circular':
        node_pos_dict = nx.circular_layout(G)
    elif layout_type == 'spring':
        node_pos_dict = nx.spring_layout(G, seed=layout_seed)
    elif layout_type == 'random':
        node_pos_dict = nx.random_layout(G, seed=layout_seed)
    elif layout_type == 'spiral':
        node_pos_dict = nx.spiral_layout(G)

    # Add the edges, along with their colors and widths
    edge_colors = []
    edge_widths = []
    for i in range(N):
        for j in range(N):
            a = A[i, j]
            if a > tol:
                G.add_edge(codes[i], codes[j])
                edge_colors.append(node_color_list[i])
                width = a * edge_size_multiple
                edge_widths.append(width)

    # Plot the networks
    nx.draw_networkx_nodes(G, 
                           node_pos_dict, 
                           node_color=node_color_list, 
                           node_size=node_sizes, 
                           edgecolors='grey', 
                           linewidths=2, 
                           alpha=0.6, 
                           ax=ax)

    nx.draw_networkx_labels(G, 
                            node_pos_dict, 
                            font_size=10, 
                            ax=ax)

    nx.draw_networkx_edges(G, 
                           node_pos_dict, 
                           edge_color=edge_colors, 
                           width=edge_widths, 
                           arrows=True, 
                           arrowsize=20, 
                           alpha=0.6,  
                           ax=ax, 
                           arrowstyle='->', 
                           node_size=node_sizes, 
                           connectionstyle='arc3,rad=0.15')

In [8]:
def spec_rad(M):
    """
    Compute the spectral radius of M.
    """
    return np.max(np.abs(np.linalg.eigvals(M)))

def eigenvector_centrality(A, k=40, authority=False):
    """
    Computes the dominant eigenvector of A. Assumes A is 
    primitive and uses the power method.  
    
    """
    A_temp = A.T if authority else A
    n = len(A_temp)
    r = spec_rad(A_temp)
    e = r**(-k) * (np.linalg.matrix_power(A_temp, k) @ np.ones(n))
    return e / np.sum(e)

In [9]:
years = [q for q in quarters if 'Q4' in q]

In [25]:
dict_data['2023-Q4']

Counterparty country,Australia,Austria,Belgium,Brazil,Canada,Chile,Denmark,Finland,France,Germany,Greece,Ireland,Italy,Japan,Korea,Mexico,Netherlands,Panama,Portugal,Spain,Sweden,Switzerland,Türkiye,United Kingdom,United States
Reporting country,,,,,,,,,,,,,,,,,,,,,,,,,
Australia,0.000,319.050,3544.908,1333.715,12196.630,54.343,721.691,589.560,22176.499,12925.570,249.236,6671.480,1739.373,47595.218,8002.767,701.864,6970.179,157.435,28.785,3848.730,1525.326,5190.051,13.328,106250.974,119814.458
Austria,1038.630,0.000,2894.715,156.585,2462.222,104.097,1559.535,1663.276,18191.379,56285.244,259.202,5562.741,6474.270,629.243,534.381,70.654,11306.189,2.138,348.514,6052.490,1883.415,8406.632,626.202,6788.190,14702.343
Belgium,1336.000,2882.000,0.000,22.000,7770.000,168.000,2219.000,1272.000,27448.000,11769.000,15.000,1975.000,3264.000,1923.000,564.000,93.000,33815.000,43.000,391.000,7334.000,979.000,1691.000,295.000,45548.000,16597.000
Brazil,8.604,392.924,187.453,0.000,651.263,34716.687,0.736,12.185,1378.485,2385.836,0.000,10.066,105.816,71.501,1685.575,3421.730,706.947,806.479,687.633,3290.129,207.360,1993.588,0.013,16861.220,25672.055
Canada,35313.979,1617.144,1720.266,0.000,0.000,0.000,1623.003,2233.569,25725.256,20136.515,0.000,21706.248,633.188,59446.614,1671.529,0.000,9618.963,0.000,0.000,3716.488,5144.922,10090.577,0.000,196925.012,1982652.332
Chile,4.312,2.822,0.232,920.288,54.488,0.000,60.397,0.000,480.901,143.689,0.000,11.757,5.901,36.329,12.077,147.205,1.454,239.183,0.566,597.434,0.022,56.932,0.000,3895.333,8612.341
Denmark,113.000,-161.000,796.000,14.000,445.000,3.000,0.000,46252.000,10937.000,19594.000,6.000,4905.000,185.000,434.000,20.000,34.000,5181.000,2.000,25.000,795.000,86503.000,1645.000,459.000,20625.000,8789.000
Finland,281.000,733.000,2641.000,0.000,2729.000,0.000,0.000,0.000,7050.000,8941.000,0.000,2858.000,150.000,326.000,140.000,7.000,2324.000,0.000,178.000,391.000,0.000,523.000,0.000,4002.000,0.000
France,31187.000,11022.000,282469.000,22387.000,58024.000,6404.000,15501.000,19757.000,0.000,231623.000,2571.000,60170.000,394425.000,292545.000,35602.000,12681.000,124927.000,5119.000,26202.000,121736.000,35777.000,79355.000,23049.000,229998.000,532987.000


In [14]:
import matplotlib.cm as cm
import networkx as nx

for qt in quarters1[3::4]:
    ecentral_hub = eigenvector_centrality(dict_data[qt], authority=False)
    ecentral_authority = eigenvector_centrality(dict_data[qt], authority=True)

    centrality = eigenvector_centrality(dict_data[qt], authority=False)
    node_colors = cm.plasma(to_zero_one_beta(centrality))
    X = to_zero_one_beta(dict_data[qt].sum(axis=1))

    countries = dict_data[qt].columns.to_list()

    fig, ax = plt.subplots(figsize=(8, 10))
    plt.axis("off")

    plot_graph(dict_data[qt].values, X, ax, countries,
               layout_type='spring',
               layout_seed=1234,
               node_size_multiple=3000,
               edge_size_multiple=0.000006,
               tol=0.0,
               node_color_list=node_colors) 
    fig.tight_layout()
    plt.savefig(f'Figures/creditflow_{qt}.pdf')
    plt.close()

## Compute centrality

In [18]:
def centrality_plot_data(countries, centrality_measures):
    df = pd.DataFrame({'country': countries,
                       'centrality':centrality_measures, 
                       'color': cm.plasma(to_zero_one_beta(centrality_measures)).tolist()
                       })
    return df.sort_values('centrality')

In [22]:
import matplotlib.patches as mpatches

for qt in quarters1[3::4]:
    
    outdegree = (dict_data[qt] > 0.0001).astype(int).sum(axis=1)
    ecentral_hub = eigenvector_centrality(dict_data[qt], authority=False)

    indegree = (dict_data[qt] > 0.0001).astype(int).sum(axis=0)
    ecentral_authority = eigenvector_centrality(dict_data[qt], authority=True)

    centrality_degree = [outdegree, indegree]
    centrality_eigen  = [ecentral_hub, ecentral_authority]

    ylabels_degree = ['out degree', 'in degree']
    ylabels_eigen  = ['eigenvector hub','eigenvector authority']

    ylims_degree = [(0, 30), (0, 30)]

    fig, axes = plt.subplots(1, 2, figsize=(10, 6))
    for i, ax in enumerate(axes):
        df = centrality_plot_data(countries, centrality_degree[i])

        ax.bar('country', 'centrality', data=df, color=df["color"], alpha=0.6)
        ax.set_xticks(range(len(df['country'])))
        ax.set_xticklabels(df['country'], rotation=90)
        patch = mpatches.Patch(color=None, label=ylabels_degree[i], visible=False)
        ax.legend(handles=[patch], fontsize=12, loc="upper left", handlelength=0, frameon=False)
        ax.set_ylim(ylims_degree[i])
    fig.tight_layout()
    plt.savefig(f'Figures/degree_centrality_{qt}.pdf')    
    plt.close()
        
    fig, axes = plt.subplots(1, 2, figsize=(10, 6))
    for i, ax in enumerate(axes):
        df = centrality_plot_data(countries, centrality_eigen[i])

        ax.bar('country', 'centrality', data=df, color=df["color"], alpha=0.6)
        ax.set_xticks(range(len(df['country'])))
        ax.set_xticklabels(df['country'], rotation=90)
        patch = mpatches.Patch(color=None, label=ylabels_eigen[i], visible=False)
        ax.legend(handles=[patch], fontsize=12, loc="upper left", handlelength=0, frameon=False)
    fig.tight_layout()
    plt.savefig(f'Figures/eigen_centrality_{qt}.pdf')    
    plt.close()